# Inference

In [1]:
import numpy as np
import random
import os
import math
from itertools import product

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
# from torch.utils.tensorboard import SummaryWriter

from torch.autograd import Variable

import torchvision.models as models
from torchvision import transforms

import albumentations as A
import albumentations.pytorch

from sklearn.metrics import accuracy_score

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [3]:
# import tensorflow as tf
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

# tf.test.is_gpu_available()

In [4]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':50,
    'PATIENCE':10,
#     'class':41 # quasi_drug
#     'class':126 # home_clean
    'class':100 # HMR 
}

In [5]:
os.listdir('./saved_models/HMR')

['RegNet_0.001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [6]:
# Train_path = '/home/lab17/jupyter_home/Data/product_image/Training/'
# Valid_path = '/home/lab17/jupyter_home/Data/product_image/Validation/'
# Test_path = '/home/lab17/jupyter_home/git/test_img/'
# save_graph_path = './result/'
# save_model_path = '/home/lab17/jupyter_home/saved_models/'

In [7]:
Train_path = './Data/product_image/Training/HMR'
Test_path = '/home/lab16/jupyter_home/git/test_img/HMR'

# Load Data

### train

In [8]:
def get_train_data(data_dir):
#     img_path_list = []
    label_list = []
    label_name_list = []
    
#     image_path = os.path.join(data_dir, 'dessert')
    image_path = data_dir
    
    for product_name in os.listdir(image_path):
        product_path = os.path.join(image_path, product_name)
        if os.path.isdir(product_path):
#             get image path
#             img_path_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_path_list.extend(glob(os.path.join(product_path, '*.png')))
            label = list(product_name.split('_')[0])
            name = product_name[6:]
            
            # get label
            label_list.append(''.join(label))
            label_name_list.append(name)
                
#     return img_path_list, label_list
    return label_list, label_name_list

In [9]:
# def get_train_data(data_dir):
#     img_path_list = []
#     label_list = []
    
# #     image_path = os.path.join(data_dir, dir_name)
#     image_path = data_dir
    
#     for product_name in os.listdir(image_path):
#         product_path = os.path.join(image_path, product_name)
#         if os.path.isdir(product_path):
#             # get image path
#             img_path_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_path_list.extend(glob(os.path.join(product_path, '*.png')))
# #             label = list(product_name[:5])
            
#             label = list(product_name.split('_')[0])
            
#             # get label
#             label_list.append(''.join(label))
                
#     return img_path_list, label_list

In [10]:
label_list, label_name_list = get_train_data(Train_path)

In [11]:
# encoder
le = preprocessing.LabelEncoder()
targets = le.fit_transform(label_list)
print('--targets\n' , targets)

label_encoder = {key:val for key, val in zip(label_list, targets)}
print(label_encoder)

--targets
 [97 39 87 64 60 47 25 54 50 34 65 19 70 27 13 20 72 29 99 98 23  7 14 38
 11 55 41 62 86 32 85 58 31  2  1 30 96 88 81 52 92 22 46 74 17 18 40  5
 61 21 91 35 49 24 69 75 77  8 28 76 44 63 33 53 90 10 94 82 89 80 42 79
 57 37 95 71 51 66 83  3 36  9 48 12 26 78 16 93 67 43 68 56 73  6  0 45
 84 15  4 59]
{'25705': 97, '15453': 39, '25428': 87, '20237': 64, '20208': 60, '15647': 47, '15266': 25, '15949': 54, '15820': 50, '15448': 34, '20256': 65, '15091': 19, '25095': 70, '15425': 27, '15028': 13, '15092': 20, '25115': 72, '15427': 29, '25707': 99, '25706': 98, '15104': 23, '10240': 7, '15029': 14, '15452': 38, '10305': 11, '16056': 55, '15456': 41, '20228': 62, '25426': 86, '15446': 32, '25425': 85, '20197': 58, '15445': 31, '10187': 2, '10185': 1, '15428': 30, '25703': 96, '25429': 88, '25421': 81, '15945': 52, '25653': 92, '15103': 22, '15478': 46, '25121': 74, '15088': 17, '15090': 18, '15455': 40, '10224': 5, '20216': 61, '15102': 21, '25608': 91, '15449': 35, '15819': 4

In [12]:
# decoder
label_decoder = {v: k for k, v in label_encoder.items()}
label_name_decoder = {key : value for key, value in zip(label_list, label_name_list)}

### validation

In [13]:
# def get_valid_data(data_dir):
#     img_valid_list = []
#     label_valid_list = []
    
#     image_path = os.path.join(data_dir, 'dessert')
    
#     for product_name in os.listdir(image_path):
#         product_path = os.path.join(image_path, product_name)
#         if os.path.isdir(product_path):
#             # get image path
#             img_valid_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_valid_list.extend(glob(os.path.join(product_path, '*.png')))
#             label = list(product_name[:5])
            
#             # get label
#             label_valid_list.append(''.join(label))
                
#     return img_valid_list, label_valid_list

In [14]:
# def valid_data_blanced(img, label):
#     x = []
#     y = []
    
#     for i in range(CFG['class']):
#         _img = img[(i * 15): ((i + 1) * 15)]
#         _label = label[i]
        
#         for img_product in _img:
#             x.append(img_product)
#             y.append(_label)
            
#     return x, y

In [15]:
# img_valid_list, label_valid_list = get_valid_data(Valid_path)
# x_valid, y_valid = valid_data_blanced(img_valid_list, label_valid_list)
# len(label_valid_list)

In [16]:
# le2 = preprocessing.LabelEncoder()
# targets_y = le2.fit_transform(y_valid)
# targets_y_t = torch.as_tensor(targets_y)
# one_hot_valid_y = F.one_hot(targets_y_t)
# one_hot_valid_y.shape

### test

In [17]:
def get_test_data(data_dir):
    img_path_list = []
    label_list = []
    
    image_path = data_dir
    
#     for product in os.listdir(image_path):

    # get image path
    img_path_list.extend(glob(os.path.join(image_path, '*.jpg')))
    img_path_list.extend(glob(os.path.join(image_path, '*.png')))
    label_list = [ip[len(Test_path):].split('_')[0][1:] for ip in img_path_list]
    # get label
#     label_list.append(''.join(label))
                
    return img_path_list, label_list

In [18]:
test_img_path, test_label_list = get_test_data(Test_path)

In [19]:
print(len(test_label_list))

224


In [20]:
# test data
test_x = test_img_path
# 레이블을 one-hot-vector로 변환
test_y = [label_encoder[key] for key in test_label_list]
test_targets = torch.as_tensor(test_y)
one_hot_test_y = F.one_hot(test_targets)

In [21]:
print([f'{i}, {y}' for i, y in zip(test_label_list, test_y)])

['15266, 25', '10245, 9', '16056, 55', '25656, 94', '15082, 16', '15888, 51', '25456, 90', '15071, 15', '15447, 33', '10185, 1', '25073, 67', '25426, 86', '25425, 85', '15424, 26', '10316, 12', '25428, 87', '15647, 47', '10184, 0', '15028, 13', '15949, 54', '25455, 89', '15455, 40', '25121, 74', '25121, 74', '25420, 80', '15476, 44', '20184, 57', '15478, 46', '20208, 60', '15451, 37', '20183, 56', '15445, 31', '15888, 51', '10241, 8', '15450, 36', '25114, 71', '25425, 85', '10261, 10', '25418, 78', '15446, 32', '25706, 98', '25653, 92', '25175, 75', '25422, 82', '15091, 19', '15949, 54', '15102, 21', '25705, 97', '10226, 6', '25706, 98', '15092, 20', '15945, 52', '15946, 53', '20256, 65', '10240, 7', '25420, 80', '25455, 89', '25425, 85', '25456, 90', '20228, 62', '20237, 64', '20197, 58', '25087, 69', '15028, 13', '25418, 78', '25608, 91', '15478, 46', '20208, 60', '15820, 50', '25703, 96', '15455, 40', '10224, 5', '20197, 58', '10316, 12', '25114, 71', '15445, 31', '25422, 82', '1545

# Model

In [22]:
class AlbumentationsCustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        
        # By default OpenCV uses BGR color space for color images,
        # so we need to convert the image to RGB color space.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.train_mode:
#             image = image.astype(np.int16)
            augmented = self.transforms(image=image)
            image = augmented['image']
            label = self.label_list[index]
            return image, label
        else:
            augmented = self.transforms(image=image)
            image = augmented['image']
            label = self.label_list[index]
            return image, label
    
    def __len__(self):
        return len(self.img_path_list)
    
A_test_transform = albumentations.Compose([
                                    A.Resize(256, 256),
#                                     A.Normalize(mean=(0.744859, 0.735139, 0.711357), std=(0.100712, 0.120692, 0.167998)),  # dessert
                                    A.Normalize(mean=(0.719834, 0.727319, 0.72394), std=(0.135879, 0.130642, 0.137573)),   # quasi_drug
#                                     A.pytorch.transforms.ToTensor(),
                                    A.pytorch.transforms.ToTensorV2(transpose_mask=True),
                                ])

# A_vali_dataset = AlbumentationsCustomDataset(x_valid, one_hot_valid_y, train_mode=True, transforms=A_test_transform)
# A_vali_loader = DataLoader(A_vali_dataset, batch_size = 5, shuffle=False, num_workers=0, collate_fn=None)

A_test_dataset = AlbumentationsCustomDataset(test_x, one_hot_test_y, train_mode=False, transforms=A_test_transform)
A_test_loader = DataLoader(A_test_dataset, batch_size = 4, shuffle=False, num_workers=0, collate_fn=None)

In [23]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        model = models.resnet50(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(2048, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000,CFG['class'])
#         self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.feature_extract(x)
        # x = x.mean(dim=(-2, -1))
        # (batch, 2048, 4, 4)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [24]:
class EfficientNetb4(torch.nn.Module):
    def __init__(self):
        super(EfficientNetb4, self).__init__()
        model = models.efficientnet_b4(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(1792, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000, CFG['class'])
#         self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.feature_extract(x)
        # (batch, 1792, 1, 1)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [25]:
class RegNet(torch.nn.Module):
    def __init__(self):
        super(RegNet, self).__init__()
        model = models.regnet_y_16gf(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(3024, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000, CFG['class'])
#         self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.feature_extract(x)
        # (batch, 3024, 1, 1)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [26]:
def score_function(real, pred):
    score = accuracy_score(real, pred)
    return score

# Inference2

In [27]:
# # sorted(os.listdir('/home/lab17/jupyter_home/saved_models/'))
# file_list = sorted(glob('/home/lab17/jupyter_home/saved_models/*.pth'))
# print(len(file_list))
# sorted(glob('/home/lab17/jupyter_home/saved_models/*.pth'))

In [28]:
# file_list=['/home/lab17/jupyter_home/saved_models/RegNet_0.001_rmsprop_CosineAnnealing_example.pth']

In [29]:
# model_dir = './data/quasi_drug/'

In [30]:
model_dir = './saved_models/HMR/'

In [31]:
os.listdir(model_dir)

['RegNet_0.001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [32]:
file_list = sorted(glob(os.path.join(model_dir,'*.pth') ))
file_list

['./saved_models/HMR/RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 './saved_models/HMR/RegNet_0.001_rmsprop_CosineAnnealing_example.pth',
 './saved_models/HMR/RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [33]:
pred_ensemble = []
batch_size = 34

device = torch.device('cpu')

for m_path in file_list:
    if 'ResNet50' in m_path:
        model_test = ResNet50().to(device)
    elif 'EfficientNetb4' in m_path:
        model_test = EfficientNetb4().to(device)
    elif 'RegNet' in m_path:
        model_test = RegNet().to(device)    

    model_test.load_state_dict(torch.load(m_path, map_location=device))
    model_test.eval()

    criterion = torch.nn.CrossEntropyLoss()

    test_loss = []
    f_pred = []

#     for img, label in tqdm(iter(A_vali_loader)):
#         img, label = img.float().to(device), label.float().to(device)

#         # Data -> Model -> Output
#         logit = model_test(img)
#         logit = torch.squeeze(logit)

#         # Calc loss
#         loss = criterion(logit, label)

#         test_loss.append(loss.item())
#         f_pred.extend(logit.argmax(1).detach().cpu().numpy().tolist())

#     print('val loss :' ,np.mean(test_loss))
#     print('val acc :', score_function(targets_y, f_pred))

#     test_loss = []
#     f_pred = []

    for img, label in tqdm(iter(A_test_loader)):
        img, label = img.float().to(device), label.float().to(device)

        # Data -> Model -> Output
        logit = model_test(img)
        logit = torch.squeeze(logit)

        # Calc loss
        loss = criterion(logit, label)

        test_loss.append(loss.item())
        f_pred.extend(logit.argmax(1).detach().cpu().numpy().tolist())
        
        
    print(m_path[len(model_dir)+2:])

    print('test loss :' ,np.mean(test_loss))
    print('test acc :', score_function(test_y, f_pred))

  0%|          | 0/56 [00:00<?, ?it/s]

gNet_0.0001_rmsprop_CosineAnnealing_example.pth
test loss : 4.32976750976273
test acc : 0.3080357142857143


  0%|          | 0/56 [00:00<?, ?it/s]

gNet_0.001_rmsprop_CosineAnnealing_example.pth
test loss : 14.852109125682286
test acc : 0.0625


  0%|          | 0/56 [00:00<?, ?it/s]

gNet_1e-05_rmsprop_CosineAnnealing_example.pth
test loss : 2.5920331930475577
test acc : 0.42857142857142855


In [34]:
def print_ox(test_y,f_pred):
    return 'o' if test_y==f_pred else 'x'
def print_name(test_y, f_pred):
    return f'{label_decoder[y]}, {label_name_decoder[label_decoder[y]]} : {label_name_decoder[label_decoder[p]]}' if test_y!=f_pred else ''
print('o/x | onehot test, pred num | img_num : pred_img_name\n')
for y, p in zip(test_y, f_pred):
    print(f'{print_ox(y,p)} | {y}, {p} | {print_name(y,p)}')
    
    
# print(f'{print_ox(y,p)} | {y}, {p} | {label_decoder[y]} : {label_name_decoder[label_decoder[p]]}')    
#     print(f'{print_ox(y,p)} {y} : {p} {label_name_decoder[y]}, {label_name_decoder[p]}')
# print(f'{test_y}, {f_pred}')
# print(f'{print_ox(test_y,f_pred)} {test_y} : {f_pred} {label_name_decoder[test_y]}, {label_name_decoder[f_pred]}')

o/x | onehot test, pred num | img_num : pred_img_name

o | 25, 25 | 
o | 9, 9 | 
o | 55, 55 | 
o | 94, 94 | 
x | 16, 69 | 15082, 백설우리밀핫케잌믹스500G : 큐원홈메이드머핀믹스300G
o | 51, 51 | 
x | 90, 95 | 25456, CJ비비고육개장500G : 백설컵전김치전210G
o | 15, 15 | 
x | 33, 7 | 15447, CJ햇반컵반강된장보리비빔밥280G : CJ햇반컵반설렁탕밥253g
x | 1, 0 | 10185, 오뚜기쇠고기스프80G : 오뚜기옥수수스프80G
x | 67, 98 | 25073, CJ비비고추어탕460G : CJ다담냉이된장찌개양념140G
x | 86, 35 | 25426, 오뚜기컵밥춘천닭갈비덮밥310G : 오뚜기컵밥햄버그덮밥310G
x | 85, 65 | 25425, 오뚜기컵밥불닭마요덮밥277g : 오뚜기고추참치
o | 26, 26 | 
x | 12, 65 | 10316, 동원살코기참치250G : 오뚜기고추참치
x | 87, 35 | 25428, 오뚜기컵밥소시지카레덮밥315G : 오뚜기컵밥햄버그덮밥310G
o | 47, 47 | 
x | 0, 15 | 10184, 오뚜기옥수수스프80G : 모리나가핫케잌믹스300G
o | 13, 13 | 
x | 54, 64 | 15949, 쉬바파우치참치와게맛살70G : 동원양반한우사골설렁국
x | 89, 7 | 25455, 청정원일상가정식장어탕450G : CJ햇반컵반설렁탕밥253g
x | 40, 18 | 15455, CJ비비고소고기미역국500G : 양반수라우거지감자탕460G
o | 74, 74 | 
o | 74, 74 | 
x | 80, 7 | 25420, CJ햇반컵반황태국밥170G : CJ햇반컵반설렁탕밥253g
x | 44, 45 | 15476, 청정원일상가정식한우진곰탕450G : 청정원일상가정식나주곰탕450G
o | 57, 57 | 
x | 46, 31 | 15478, CJ비비